In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
data = pd.read_csv("/kaggle/input/lending-club-data/loan_data.csv", low_memory=False)
data.head()

In [ ]:
data['purpose'].value_counts()

In [ ]:
data['purpose'] = data['purpose'].map({'debt_consolidation':0,
                                             'all_other':1,'credit_card':2,
                                             'home_improvement' : 3,
                                             'small_business' : 4,
                                             'major_purchase' : 5,
                                             'educational' : 6,
                                            })

In [ ]:
from sklearn.model_selection import train_test_split
 
x = data.loc[:,'credit.policy':'pub.rec']
y = data['not.fully.paid']
 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
check = pd.DataFrame(data=x)
check.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import statsmodels.api as sm
 
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)
 
x2 = sm.add_constant(x)
model = sm.OLS(y, x2)
result = model.fit()
print(result.summary())
 
y_pred = log_reg.predict(x_test)
print('Accuracy :', metrics.accuracy_score(y_test, y_pred))

In [ ]:
result.params

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
cnf_metrix = metrics.confusion_matrix(y_test, y_pred)
print(cnf_metrix)
 
class_names = ['Fully Paid', 'Not Fully Paid']
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
 

sns.heatmap(pd.DataFrame(cnf_metrix), annot=True, cmap='YlGnBu', fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title("Confusion matrix", y=1.1)
plt.ylabel("Actual label")
plt.xlabel("Predict label")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score,recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix,precision_recall_curve, roc_curve, auc

def get_clf_eval(y_test=None, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('Acuuracy: {0:.4f}, Precision: {1:.4f}, Recall: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    
get_clf_eval(y_test,y_pred)

In [ ]:
import matplotlib.pyplot as plt

fpr, tpr, threshold = roc_curve(y_train, log_reg.decision_function(x_train))

plt.plot(fpr, tpr, 'o-', ms = 2, label="Log Reg")
plt.legend
plt.plot([0,1],[0,1], 'k--', label="abs")
plt.xlabel('Spec')
plt.ylabel('Recall')
plt.title('ROC Curve')
plt.show()

print('AUC : {}'.format(auc(fpr, tpr)))